In [ ]:
#default_exp peptide.mobility

In [ ]:
#export
import numpy as np
import pandas as pd

from alphabase.peptide.fragment import update_precursor_mz
from alphabase.constants.element import common_const_dict

CCS_IM_COEF = common_const_dict['MOBILITY']['CCS_IM_COEF']
IM_GAS_MASS = common_const_dict['MOBILITY']['IM_GAS_MASS']

def get_reduced_mass(precursor_mzs, charges):
    reduced_masses = precursor_mzs*charges
    return reduced_masses*IM_GAS_MASS/(reduced_masses+IM_GAS_MASS)

def ccs_to_mobility_bruker(ccs_values, charges, precursor_mzs):
    reduced_masses = get_reduced_mass(precursor_mzs, charges)
    return ccs_values*np.sqrt(reduced_masses)/charges/CCS_IM_COEF

def mobility_to_ccs_bruker(im_values, charges, precursor_mzs):
    reduced_masses = get_reduced_mass(precursor_mzs, charges)
    return im_values*charges*CCS_IM_COEF/np.sqrt(reduced_masses)

def ccs_to_mobility_bruker_for_df(
    precursor_df:pd.DataFrame,
    ccs_column:str,
)->pd.DataFrame:
        if 'precursor_mz' not in precursor_df.columns:
            precursor_df = update_precursor_mz(precursor_df)
        return ccs_to_mobility_bruker(
            precursor_df[ccs_column].values, 
            precursor_df.charge.values,
            precursor_df.precursor_mz.values
        )

def mobility_to_ccs_bruker_for_df(
    precursor_df:pd.DataFrame,
    mobility_column:str,
)->pd.DataFrame:
    if 'precursor_mz' not in precursor_df.columns:
        precursor_df = update_precursor_mz(precursor_df)
    return mobility_to_ccs_bruker(
        precursor_df[mobility_column].values,
        precursor_df.charge.values,
        precursor_df.precursor_mz.values
    )